### Phase 4 (Modeling)

_TODO: YOUR ANSWER WILL IN THIS DOCUMENT_

Linkin notebook käsittelee autojen hintojen ennustamista syväoppimismallilla ja sen optimointia neuroverkkorakenteiden avulla. Mallin kouluttamiseen on käytetty GPU:n laskentatehoa.

```
Mean Squared Error (MSE): 7562431.5000
Root Mean Squared Error (RMSE): 2749.9875
Mean Absolute Error (MAE): 1987.2920
R-squared (R2): 0.9504
```

[DL-results notebook](https://gitlab.labranet.jamk.fi/AC7766/aida-projekti-syksy-2024-ryhma-1/-/blob/main/docs/Phase%204/dlearn_results.ipynb)

Linkin notebook sisältää autojen hintojen ennustamiseen optimoidun Random Forest -mallin ja sen suorituskykyanalyysin.

```
Mean Squared Error: 3314424.73
R-squared: 0.98
Root Mean Squared Error: 1820.56
MAE: 1172.4659546276207
```

[RF-results notebook](https://gitlab.labranet.jamk.fi/AC7766/aida-projekti-syksy-2024-ryhma-1/-/blob/main/docs/Phase%204/rf_results.ipynb)

## Knn algo setit

In [2]:
X = knn_df.drop(columns=['price'])
y = knn_df['price']


NameError: name 'knn_df' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize KNN regressor
knn = KNeighborsRegressor(n_neighbors=5)

# Train the model
knn.fit(X_train, y_train)

# Predict
y_pred = knn.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': [3, 5, 7, 10, 15, 20]}
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best R-squared:", grid_search.best_score_)


In [ ]:
knn = KNeighborsRegressor(n_neighbors=7)

# Train the model
knn.fit(X_train, y_train)

# Predict
y_pred = knn.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

## Tree based algo setit

In [ ]:
# testataan suoraan vaan näillä seteillä
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


# Split data
X = rf_df.drop(columns=['price'])  # Features
y = rf_df['price']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# model train
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
# Evaluate the model
y_pred = rf_model.predict(X_test)
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

In [ ]:
# testataan feature importance rf-mallille

import matplotlib.pyplot as plt
feature_importances = rf_model.feature_importances_
sorted_idx = feature_importances.argsort()
plt.figure(figsize=(8, 14)) 
plt.barh(X_train.columns[sorted_idx], feature_importances[sorted_idx])
plt.xlabel("Random Forest Feature Importance")
plt.ylabel("Features")
plt.title("Feature Importance in Random Forest Model")
plt.show()

## Deep learning algo setit


In [ ]:
# Define target and features
target = 'price'
X = tf_df.drop(columns=[target])
y = tf_df[target]

# Split into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# Define embedding dimensions (rule of thumb: min(50, number of unique categories // 2))
embedding_dims = {
    'fuel_type_encoded': (5, 3),   # (unique_categories, embedding_size)
    'make_encoded': (48, 24),
    'city_encoded': (411, 50),
    'model_encoded': (657, 50)
}

# Create inputs and embedding layers
inputs = {}
embeddings = []

for col, (num_categories, embed_size) in embedding_dims.items():
    inp = Input(shape=(1,), name=col)
    emb = Embedding(input_dim=num_categories, output_dim=embed_size, input_length=1)(inp)
    embeddings.append(Flatten()(emb))
    inputs[col] = inp

In [ ]:
# Numerical features
numerical_cols = X_train.drop(columns=embedding_dims.keys()).columns
numerical_input = Input(shape=(len(numerical_cols),), name='numerical_inputs')

# Add to inputs
inputs['numerical'] = numerical_input

In [ ]:
# Combine embeddings and numerical inputs
all_features = Concatenate()([*embeddings, numerical_input])

In [ ]:
# Add dense layers
#x = Dense(128, activation='relu')(all_features)
x = Dense(128, activation='relu')(all_features)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)

# Output layer (predicting price)
output = Dense(1, activation='linear', name='output')(x)

# Build and compile model
model = Model(inputs=list(inputs.values()), outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print model summary
model.summary()

In [ ]:
# Prepare input dictionaries for TensorFlow
train_inputs = {col: X_train[col] for col in embedding_dims.keys()}
train_inputs['numerical_inputs'] = X_train[numerical_cols].values  # Correct key name

test_inputs = {col: X_test[col] for col in embedding_dims.keys()}
test_inputs['numerical_inputs'] = X_test[numerical_cols].values  # Correct key name


In [ ]:
# Train the model

history = model.fit(
    train_inputs,
    y_train,
    validation_data=(test_inputs, y_test),
    epochs=40,
    batch_size=64
)

# Evaluate the model
loss, mae = model.evaluate(test_inputs, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")